# Loading the pre-processed data

In [2]:
import pandas as pd
# Load the preprocessed dataset
df = pd.read_csv(r"C:\Users\rajak\Downloads\AI Internship\clinical_notes_preprocessed_no_spellcheck.csv")
# Display basic information
print(df.shape)
df.head()

(158114, 3)


,patient_id,task,note_preprocessed
0,0,Paraphrasing,discharge summary patient year old male modera...
1,1,Coreference Resolution,discharge summary admission date insert date d...
2,2,Summarization,hospital course summary admission date insert ...
3,3,Relation Extraction,discharge summary patient year old male hospit...
4,4,Relation Extraction,discharge summary patient information name red...


In this case we loads the existing preprocessed data and printed his for the visualisation of the data.

# Raw pre-processed text

In [4]:
# Extract preprocessed clinical notes
texts = df["note_preprocessed"].astype(str).tolist()
# Display one sample text
print("Sample preprocessed clinical note:\n")
print(texts[0])

Sample preprocessed clinical note:

discharge summary patient year old male moderate acute respiratory distress syndrome covid hospital course patient admitted hospital symptom fever dry cough dyspnea physical therapy acute ward patient experienced coughing attack induced oxygen desaturation dyspnea change position deep breathing avoid rapid deterioration respiratory failure step step approach used position change breathing exercise adapted avoid prolonged coughing oxygen desaturation close monitoring patient managed perform strength walking exercise low level exercise progression low initially increased daily hospital discharge rehabilitation clinic day clinical outcome patient discharged day rehabilitation clinic making satisfactory progress symptom resolved follow patient receive follow care rehabilitation clinic regular monitoring progress rehabilitation exercise full recovery new symptom concern reported clinic immediately overall impression patient responded well treatment approp

displayed the pre-processed data. That needs to be embeded.

# clinical text to vector- TF-IDF

In [6]:
pip install sentence-transformers

   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/10.3 MB 7.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.3 MB 8.4 MB/s eta 0:00:01
   ------------------- -------------------- 5.0/10.3 MB 9.1 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.3 MB 9.1 MB/s eta 0:00:01
   --------------------------------- ------ 8.7/10.3 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  10.2/10.3 MB 9.0 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.3 MB 8.4 MB/s  0:00:01
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   -------------------------------------- - 2.6/2.7 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 9.3 MB/s  0:00:00
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   - -------------------------------------- 3.9/113.8 MB 29.4 MB/s eta 0:00:04
   --- ----------------

downloading the essential libraries for converting the clinical text to the vector.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2)
)

using TF-IDF (Basic model) to convert the text terms to the vectors.

In [10]:
tfidf_vectors = vectorizer.fit_transform(texts)
print("TF-IDF matrix shape:", tfidf_vectors.shape)

TF-IDF matrix shape: (158114, 3000)


Converted the text format to the vector format for the further comptation